# 1. Importar los datos

Debemos combinar las etiquetas (labels.csv) con los datos preprocesados (preprocessed_dataset.csv) basándonos en la columna respondent_id.

In [1]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.multioutput import MultiOutputClassifier
import pandas as pd
import numpy as np

c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X_train = pd.read_csv("../../Data/preprocessed_X_train.csv", index_col="respondent_id")
X_test = pd.read_csv("../../Data/preprocessed_X_test.csv", index_col="respondent_id")
y_train = pd.read_csv("../../Data/preprocessed_y_train.csv", index_col="respondent_id")
y_test = pd.read_csv("../../Data/preprocessed_y_test.csv", index_col="respondent_id")

# 2. Dividir los datos en conjuntos de entrenamiento y prueba

Utilizamos train_test_split para separar los datos.

# 3. Entrenar un árbol de decisión

Utilizamos DecisionTreeClassifier de Scikit-Learn.

In [3]:
def objective(trial):
    # Espacio de búsqueda de hiperparámetros
    param_grid = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 5, 10, step=1),  # Reducir rango
        'min_samples_split': trial.suggest_int('min_samples_split', 5, 20, step=1),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 25, step=1),
        'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.1, step=0.01),  # Reducir rango
        'splitter': trial.suggest_categorical('splitter', ['best', 'random']),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced'])
    }

    # Modelo base
    cart_model = DecisionTreeClassifier(random_state=42, **param_grid)
    multi_cart = MultiOutputClassifier(cart_model)

    # Validación cruzada
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in skf.split(X_train, y_train['h1n1_vaccine']):  # Usa una etiqueta para dividir
        # Dividir los datos
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Entrenar y evaluar el modelo
        multi_cart.fit(X_train_fold, y_train_fold)
        y_proba = multi_cart.predict_proba(X_val_fold)

        # Calcular AUROC para cada etiqueta
        roc_auc_h1n1 = roc_auc_score(y_val_fold['h1n1_vaccine'], y_proba[0][:, 1])
        roc_auc_seasonal = roc_auc_score(y_val_fold['seasonal_vaccine'], y_proba[1][:, 1])
        scores.append(np.mean([roc_auc_h1n1, roc_auc_seasonal]))

    return np.mean(scores)

In [4]:
# Configurar Optuna
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)

# Ejecutar Optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=200, show_progress_bar=True)

[I 2025-01-21 19:43:22,844] A new study created in memory with name: no-name-87fc1999-df79-4e64-9cfa-dae7381d59fc
Best trial: 134. Best value: 0.822914: 100%|██████████| 200/200 [01:57<00:00,  1.71it/s]


In [53]:
# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", study.best_params)
print("Mejor puntuación AUROC:", study.best_value)

# Entrenar el modelo final con los mejores hiperparámetros
best_params = study.best_params
cart_model = DecisionTreeClassifier(random_state=42, **best_params)
multi_cart = MultiOutputClassifier(cart_model)
multi_cart.fit(X_train, y_train)

Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 7, 'min_samples_split': 20, 'min_samples_leaf': 23, 'max_features': None, 'min_impurity_decrease': 0.0, 'splitter': 'best', 'class_weight': None}
Mejor puntuación AUROC: 0.8368389327414809


MultiOutputClassifier(estimator=DecisionTreeClassifier(max_depth=7,
                                                       min_samples_leaf=23,
                                                       min_samples_split=20,
                                                       random_state=42))

# 4. Evaluar el modelo

Calculamos las métricas de rendimiento como la precisión, el recall, el F1-score y la exactitud.

In [54]:
# Evaluar el modelo en el conjunto de prueba
y_proba = multi_cart.predict_proba(X_test)

# Calcular AUROC para cada etiqueta
roc_auc_h1n1 = roc_auc_score(y_test['h1n1_vaccine'], y_proba[0][:, 1])
roc_auc_seasonal = roc_auc_score(y_test['seasonal_vaccine'], y_proba[1][:, 1])

print(f"AUROC para H1N1: {roc_auc_h1n1}")
print(f"AUROC para vacuna estacional: {roc_auc_seasonal}")

# Reporte de clasificación
y_pred = multi_cart.predict(X_test)
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

AUROC para H1N1: 0.8498549197948876
AUROC para vacuna estacional: 0.8363248005281694
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.47      0.56      1135
           1       0.76      0.71      0.74      2488

   micro avg       0.74      0.64      0.69      3623
   macro avg       0.73      0.59      0.65      3623
weighted avg       0.74      0.64      0.68      3623
 samples avg       0.34      0.32      0.32      3623



c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VEN

# 5. Visualizar el árbol de decisión (opcional)

Podemos visualizar el árbol para comprender cómo toma decisiones.

# 6. Archivo de envío a la competición

Cargamos el dataset de prueba de la competición para obtener los resultados del modelo.

In [55]:
# Cargar el dataset de prueba preprocesado
test_set_preprocessed = pd.read_csv("../../Data/preprocessed_test_simple_dataset.csv")

# Asegurarse de que respondent_id esté disponible
respondent_ids = test_set_preprocessed['respondent_id']

# Eliminar respondent_id del conjunto de características
X_test_final = test_set_preprocessed.drop(columns=['respondent_id'])

# Realizar predicciones de probabilidad
y_proba_test = multi_cart.predict_proba(X_test_final)

# Extraer las probabilidades para la clase positiva (1)
y_proba_h1n1_test = y_proba_test[0][:, 1]  # Probabilidades para H1N1
y_proba_seasonal_test = y_proba_test[1][:, 1]  # Probabilidades para vacuna estacional

# Crear el DataFrame de submission
submission = pd.DataFrame({
    "respondent_id": respondent_ids,
    "h1n1_vaccine": y_proba_h1n1_test,
    "seasonal_vaccine": y_proba_seasonal_test
})

# Guardar el archivo de submission
submission.to_csv("Submissions/DecisionTree_submission.csv", index=False)
print("Archivo 'DecisionTree_submission.csv' generado correctamente.")

Archivo 'DecisionTree_submission.csv' generado correctamente.
